In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil

In [2]:
!kaggle datasets download -d eeshawn/flickr30k

!unzip flickr30k.zip

Streaming output truncated to the last 5000 lines.
  inflating: flickr30k_images/5354811242.jpg  
  inflating: flickr30k_images/5354904685.jpg  
  inflating: flickr30k_images/5354911151.jpg  
  inflating: flickr30k_images/535529555.jpg  
  inflating: flickr30k_images/5356236664.jpg  
  inflating: flickr30k_images/5356318668.jpg  
  inflating: flickr30k_images/535830521.jpg  
  inflating: flickr30k_images/5359437945.jpg  
  inflating: flickr30k_images/5361355399.jpg  
  inflating: flickr30k_images/53614287.jpg  
  inflating: flickr30k_images/536147209.jpg  
  inflating: flickr30k_images/5361551308.jpg  
  inflating: flickr30k_images/536156200.jpg  
  inflating: flickr30k_images/5362687336.jpg  
  inflating: flickr30k_images/5363528971.jpg  
  inflating: flickr30k_images/5363903246.jpg  
  inflating: flickr30k_images/5364849864.jpg  
  inflating: flickr30k_images/536495604.jpg  
  inflating: flickr30k_images/5365075444.jpg  
  inflating: flickr30k_images/536533294.jpg  
  inflating: flic

In [3]:
!wget https://raw.githubusercontent.com/Ramkanc/IIITHgrp20/refs/heads/main/Flickr30k.csv

--2025-02-22 10:33:30--  https://raw.githubusercontent.com/Ramkanc/IIITHgrp20/refs/heads/main/Flickr30k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12425106 (12M) [text/plain]
Saving to: ‘Flickr30k.csv’

Flickr30k.csv       100%[===================>]  11.85M  --.-KB/s    in 0.1s    

2025-02-22 10:33:30 (114 MB/s) - ‘Flickr30k.csv’ saved [12425106/12425106]



In [4]:
image_data_location = "/content/flickr30k_images"
caption_data_location = "/content/Flickr30k.csv"

In [5]:
# Collect images from image_data_location folder which are getting opened
images_good = []
for filename in os.listdir(image_data_location):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        try:
            img = Image.open(os.path.join(image_data_location, filename))
            images_good.append(filename.split(".")[0]) #append to a list
        except IOError:
            print("Unable to open image:", filename)

print(f"Loaded {len(images_good)} images")

Loaded 31783 images


In [6]:
raw_df = pd.read_csv(caption_data_location, sep=",", header=0)#, names=['image','caption'])

In [7]:
# Clean image names
raw_df['image'] = raw_df['image'].astype(str).str.strip()

raw_df.head()

image                                            caption
0  1000092795   Two young guys with shaggy hair look at their...
1  1000092795   Two young , White males are outside near many...
2  1000092795   Two men in green shirts are standing in a yard .
3  1000092795       A man in a blue shirt standing in a garden .
4  1000092795            Two friends enjoy time spent together .

In [8]:
# Find df[image] rows not in images_good list
indices_not_in_list = []
for index, image_name in raw_df['image'].items():
    if image_name not in images_good:
        print(image_name)
        indices_not_in_list.append(index)

In [4]:
df = raw_df.drop(indices_not_in_list, inplace=False)
print(df.head())
print(f"raw df shape - {raw_df.shape}")
print(f"new df shape - {df.shape}")


NameError: name 'raw_df' is not defined

In [14]:
# Find the maximum words string from df['captions'] column
df['word_count'] = df['caption'].apply(lambda x: len(x.split()))
max_words_string = df.loc[df['word_count'].idxmax(), 'caption']
avg_words = df['word_count'].mean()
print(f"The string with the maximum words is:\n{max_words_string}")
print(f"the length of the string is {len(max_words_string.split())}")
print(f"The average length of the string is {avg_words}")

The string with the maximum words is:
 A man wearing a helmet , red pants with white stripes going down the sides and a white and red shirt is on a small bicycle using only his hands while his legs are up in the air , while another man wearing a light blue shirt with dark blue trim and black pants with red stripes going up the sides is standing nearby , gesturing toward the first man and holding a small figurine of one of the seven dwarves .
the length of the string is 82
The average length of the string is 13.389418175868709


In [3]:
data_idx = 11
image_name = df.iloc[data_idx,0]
image_path = image_data_location + "/" + image_name+".jpg"
print(image_path)
img = mpimg.imread(image_path)
plt.figure(figsize=(3, 3))
plt.axis('off')
plt.imshow(img)
plt.show()

NameError: name 'df' is not defined

In [16]:
for i in range(data_idx, data_idx+5):
    print(f"Caption - {df.iloc[i,1]}")

Caption -  A little girl in a pink dress going into a wooden cabin .
Caption -  A little girl climbing the stairs to her playhouse .
Caption -  A little girl climbing into a wooden playhouse 
Caption -  A girl going into a wooden building .
Caption -  Someone in a blue shirt and hat is standing on stair and leaning against a window .


In [17]:
# Preprocess dataset
image_folder = image_data_location
dataframe = df.drop(columns=['word_count'])

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
embed_dim = 512  # Embedding dimension of CLIP
hidden_dim = 512  # Hidden dimension of LSTM
tokenizer = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32").tokenizer
vocab_size = tokenizer.vocab_size
print(f"length of tokenizer: {vocab_size}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

length of tokenizer: 49408


In [21]:
def tranform_img_caption(image_folder, dataframe, tokenizer, device):
  images_tensor = []
  captions_tensor = []
  # Image Transform
  transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])
  for image_name, caption in zip(dataframe['image'], dataframe['caption']):
    img_path = os.path.join(image_folder, image_name + ".jpg")
    img = Image.open(img_path).convert('RGB')
    img = transform(img)
    images_tensor.append(img)

    tokenized_caption = tokenizer(caption, return_tensors="pt", padding='max_length', truncation=True, max_length=16).input_ids.squeeze()
    captions_tensor.append(tokenized_caption)

  images_tensor = torch.stack(images_tensor).to(device)
  captions_tensor = torch.stack(captions_tensor).to(device)
  return [images_tensor, captions_tensor]

In [2]:
images_tensor, captions_tensor = tranform_img_caption(image_folder, dataframe, tokenizer, device)

NameError: name 'tranform_img_caption' is not defined

In [ ]:
print(f"Shape of captions_tensor: {captions_tensor.shape}")
print(f"Shape of images_tensor: {images_tensor.shape}")

In [ ]:
# Dataset Class
class ImageCaptionDataset(Dataset):
    def __init__(self, images_tensor, captions_tensor):
        self.images_tensor = images_tensor
        self.captions_tensor = captions_tensor

    def __len__(self):
        return len(self.images_tensor)

    def __getitem__(self, idx):
        image = self.images_tensor[idx].to(torch.float32)
        caption = self.captions_tensor[idx]
        return image, caption

In [ ]:
dataset = ImageCaptionDataset(images_tensor, captions_tensor)

In [ ]:
# Split the dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size]
)

# Extract test dataset indexes
test_indexes = test_dataset.indices

# Create data loaders
batch_size = 32  # Adjust batch size as needed

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Example usage of data loaders (optional)
for images, captions in train_loader:
    print(images.shape, captions.shape)
    break

In [ ]:
# Encoder: CLIP
class CLIPEncoder(nn.Module):
    def __init__(self, model_name="openai/clip-vit-base-patch32"):
        super(CLIPEncoder, self).__init__()
        self.clip_model = CLIPModel.from_pretrained(model_name)
        self.processor = CLIPProcessor.from_pretrained(model_name)

    def forward(self, images):
        with torch.no_grad():
            image_features = self.clip_model.get_image_features(images)
        return image_features

In [ ]:
# Attention Mechanism for Encoder-Decoder
# class Attention(nn.Module):
#     def __init__(self, hidden_size):
#         super(Attention, self).__init__()
#         self.attn = nn.Linear(hidden_size * 2, hidden_size)
#         self.v = nn.Linear(hidden_size, 1, bias=False)

#     def forward(self, hidden, encoder_outputs):
#         # hidden: [batch_size, hidden_size]
#         # encoder_outputs: [batch_size, seq_len, hidden_size]

#         hidden = hidden.unsqueeze(1).repeat(1, encoder_outputs.size(1), 1)  # [batch_size, seq_len, hidden_size]

#         combined = torch.cat((hidden, encoder_outputs), dim=2)

#         energy = torch.tanh(self.attn(combined))  # [batch_size, seq_len, hidden_size]
#         attention = self.v(energy).squeeze(2)  # [batch_size, seq_len]

#         return F.softmax(attention, dim=1)

In [ ]:
# Decoder: LSTM without Attention
class Decoder(nn.Module):
    def __init__(self, embed_dim, hidden_dim, vocab_size, num_layers=1):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim + hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, features, captions):
        embeddings = self.embedding(captions)
        features = features.unsqueeze(1).repeat(1, captions.size(1), 1)  # Expand features to match caption length
        lstm_input = torch.cat((embeddings, features), dim=2)
        output, _ = self.lstm(lstm_input)
        outputs = self.fc(output)
        return outputs

In [ ]:
# Decoder: LSTM with Encoder-Decoder Attention
# class DecoderWithAttention(nn.Module):
#     def __init__(self, embed_dim, hidden_dim, vocab_size, num_layers=1):
#         super(DecoderWithAttention, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim)
#         self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
#         self.attention = Attention(hidden_dim)
#         self.fc = nn.Linear(hidden_dim * 2, vocab_size)

#     def forward(self, features, captions):
#         embeddings = self.embedding(captions)

#         hidden_states = []
#         output, (hidden, cell) = self.lstm(embeddings)

#         context = []
#         for t in range(output.size(1)):
#             attn_weights = self.attention(hidden[-1], features.unsqueeze(1))  # features expanded for batch processing
#             context_vector = torch.bmm(attn_weights.unsqueeze(1), features.unsqueeze(1)).squeeze(1)
#             context.append(context_vector)

#         context = torch.stack(context, dim=1)  # [batch_size, seq_len, hidden_size]

#         combined = torch.cat((output, context), dim=2)  # [batch_size, seq_len, hidden_size * 2]

#         outputs = self.fc(combined)
#         return outputs

In [ ]:
# Combined Model
class ImageCaptioningModel(nn.Module):
    def __init__(self, embed_dim, hidden_dim, vocab_size):
        super(ImageCaptioningModel, self).__init__()
        self.encoder = CLIPEncoder()
        self.decoder = Decoder(embed_dim, hidden_dim, vocab_size)

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs = self.decoder(features, captions)
        return outputs

In [ ]:
# Combined Model
# class ImageCaptioningModel(nn.Module):
#     def __init__(self, embed_dim, hidden_dim, vocab_size):
#         super(ImageCaptioningModel, self).__init__()
#         self.encoder = CLIPEncoder()
#         self.decoder = DecoderWithAttention(embed_dim, hidden_dim, vocab_size)

#     def forward(self, images, captions):
#         features = self.encoder(images)
#         outputs = self.decoder(features, captions)
#         return outputs

In [ ]:
# Initialize the model
model = ImageCaptioningModel(embed_dim, hidden_dim, vocab_size).to(device)

In [ ]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def generate_caption_beam_search(image_tensor, beam_size=3):
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.unsqueeze(0).to(device)

        # Initialize beam search
        start_token = torch.tensor([tokenizer.bos_token_id]).to(device)
        sequences = [[start_token, 0.0]]  # (sequence, log_probability)

        for _ in range(20):  # Maximum caption length
            all_candidates = []
            for seq, score in sequences:
                inputs = seq.unsqueeze(0)
                outputs = model(image_tensor, inputs)
                probs = F.softmax(outputs[:, -1], dim=-1)  # Probabilities for next word

                # Get top k probabilities and their indices
                top_k_probs, top_k_indices = probs.topk(beam_size)

                for prob, index in zip(top_k_probs[0], top_k_indices[0]):
                    candidate = [torch.cat([seq, index.unsqueeze(0)]), score + torch.log(prob).item()]
                    all_candidates.append(candidate)

            # Order candidates by score and select top k
            ordered = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)
            sequences = ordered[:beam_size]

            # Check for EOS token
            best_sequence = sequences[0][0]
            if best_sequence[-1].item() == tokenizer.eos_token_id:
                break

        # Decode the best sequence
        decoded_caption = tokenizer.decode(best_sequence, skip_special_tokens=True)
        return decoded_caption

In [ ]:
# Lists to store training and validation losses
train_losses = []
val_losses = []
# Adjust Training Loop for Matching Dimensions
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    epoch_train_loss = 0  # Accumulate loss for the current epoch
    for images, captions in train_loader:
        images, captions = images.to(device), captions.to(device)

        inputs = captions[:, :-1]  # Exclude the last token for input
        targets = captions[:, 1:]  # Exclude the first token for target

        optimizer.zero_grad()
        outputs = model(images, inputs)

        outputs = outputs.view(-1, outputs.size(-1))
        targets = targets.reshape(-1)

        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        epoch_train_loss += loss.item()  # Add batch loss to epoch loss

    train_losses.append(epoch_train_loss / len(train_loader))  # Append average epoch loss

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_train_loss / len(train_loader):.4f}")

    # Validation Loop
    model.eval()
    epoch_val_loss = 0  # Accumulate loss for the current epoch
    with torch.no_grad():
        for images, captions in val_loader:
            images, captions = images.to(device), captions.to(device)
            inputs = captions[:, :-1]
            targets = captions[:, 1:]
            outputs = model(images, inputs)
            outputs = outputs.view(-1, outputs.size(-1))
            targets = targets.reshape(-1)
            loss = criterion(outputs, targets)
            epoch_val_loss += loss.item()  # Add batch loss to epoch loss

    val_losses.append(epoch_val_loss / len(val_loader))  # Append average epoch loss

    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {epoch_val_loss / len(val_loader):.4f}")

Epoch 1/20, Train Loss: 2.8700
Epoch 1/20, Validation Loss: 2.4040
Epoch 2/20, Train Loss: 2.1167
Epoch 2/20, Validation Loss: 2.2096
Epoch 3/20, Train Loss: 1.8177
Epoch 3/20, Validation Loss: 2.1368
Epoch 4/20, Train Loss: 1.5883
Epoch 4/20, Validation Loss: 2.1224
Epoch 5/20, Train Loss: 1.3979
Epoch 5/20, Validation Loss: 2.1340
Epoch 6/20, Train Loss: 1.2378
Epoch 6/20, Validation Loss: 2.1591
Epoch 7/20, Train Loss: 1.0995
Epoch 7/20, Validation Loss: 2.2049
Epoch 8/20, Train Loss: 0.9786
Epoch 8/20, Validation Loss: 2.2614
Epoch 9/20, Train Loss: 0.8716
Epoch 9/20, Validation Loss: 2.3231
Epoch 10/20, Train Loss: 0.7794
Epoch 10/20, Validation Loss: 2.3805
Epoch 11/20, Train Loss: 0.6987
Epoch 11/20, Validation Loss: 2.4491
Epoch 12/20, Train Loss: 0.6293
Epoch 12/20, Validation Loss: 2.5328
Epoch 13/20, Train Loss: 0.5685
Epoch 13/20, Validation Loss: 2.6027
Epoch 14/20, Train Loss: 0.5179
Epoch 14/20, Validation Loss: 2.6855
Epoch 15/20, Train Loss: 0.4732
Epoch 15/20, Validat

In [ ]:
# provide code to save the model as pickle file
import pickle

In [ ]:
# prompt: provide code to save the model as pickle file

#import pickle

# Save the model
with open('Clip_lstm_noat_8_20.pkl', 'wb') as f:
    pickle.dump(model, f)


In [ ]:
# Assuming 'model' is th trained  instance
torch.save(model.state_dict(), 'Clip_lstm_noat_8_20.pth')

In [ ]:
print (train_losses)
print (val_losses)
print (len(train_losses))
print (len(val_losses))

[2.8700054045251235, 2.116712182053464, 1.8177316867315723, 1.5883447289938042, 1.3978834816589658, 1.2377533112236634, 1.0995106504486483, 0.9786072727366398, 0.8715763323509647, 0.7794409239362822, 0.6987411465805039, 0.6293204543383225, 0.5685421830050559, 0.5179346700018573, 0.47317165398432803, 0.4370517873540226, 0.40592485371785675, 0.3781173735444725, 0.35611286132114206, 0.3379410933040583]
[2.4040026683507003, 2.209638005166542, 2.1367950739822987, 2.12242352493166, 2.1340015187976866, 2.1591414766987476, 2.204889961115019, 2.2614404075727688, 2.323070474496977, 2.3804735743154692, 2.4490529280009232, 2.5327752077673362, 2.6027101118733564, 2.685512960426451, 2.7443758052165115, 2.8195320959166277, 2.885385060873557, 2.949443332792267, 3.0091381598645306, 3.0708103630486434]
20
20


In [ ]:

# Create a DataFrame to store the training and validation losses
epochs = range(1, num_epochs + 1)
loss_df = pd.DataFrame({
    'Epoch': epochs,
    'Train Loss': train_losses,
    'Validation Loss': val_losses
})

# Print or save the DataFrame
loss_df



Epoch  Train Loss  Validation Loss
0       1    2.870005         2.404003
1       2    2.116712         2.209638
2       3    1.817732         2.136795
3       4    1.588345         2.122424
4       5    1.397883         2.134002
5       6    1.237753         2.159141
6       7    1.099511         2.204890
7       8    0.978607         2.261440
8       9    0.871576         2.323070
9      10    0.779441         2.380474
10     11    0.698741         2.449053
11     12    0.629320         2.532775
12     13    0.568542         2.602710
13     14    0.517935         2.685513
14     15    0.473172         2.744376
15     16    0.437052         2.819532
16     17    0.405925         2.885385
17     18    0.378117         2.949443
18     19    0.356113         3.009138
19     20    0.337941         3.070810

In [ ]:
loss_df.to_csv('train_val_clip_lstm_noatt_8.csv', index=False) #Optional: save to file

In [ ]:
# Calculate and print the average training and validation losses
avg_train_loss = sum(train_losses) / len(train_losses) if train_losses else 0
avg_val_loss = sum(val_losses) / len(val_losses) if val_losses else 0

print(f"Average Training Loss: {avg_train_loss:.4f}")
print(f"Average Validation Loss: {avg_val_loss:.4f}")

Average Training Loss: 0.9780
Average Validation Loss: 2.5042


In [ ]:
# Install necessary libraries with version numbers
!pip install nltk==3.8.1
!pip install rouge-score==0.1.2
!pip install jiwer==3.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.0 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.9.1
    Uninstalling nltk-3.9.1:
      Successfully uninstalled nltk-3.9.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c40a127fc408640a5c51cf987bd91a2ef7372d81ad680cb8b0f522ab9ef5fff4
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.6 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import jiwer
import matplotlib.pyplot as plt # Import for plotting
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
print(test_indexes)

[18152, 34473, 13507, 28090, 6569, 11779, 38315, 13744, 40242, 22043, 40352, 30033, 34225, 15614, 10181, 5236, 25470, 30719, 4273, 35171, 23285, 19331, 13571, 34462, 7167, 8198, 31123, 21399, 29267, 21377, 26714, 20247, 19182, 18109, 4480, 13680, 18026, 36173, 6958, 25456, 36166, 423, 12677, 37623, 2199, 23870, 4946, 1079, 1254, 5520, 35448, 5941, 5661, 20566, 4793, 26236, 34870, 6813, 5308, 17632, 4974, 26058, 5952, 19597, 8949, 11286, 38245, 35701, 15881, 11114, 36091, 22093, 29612, 22720, 10777, 36119, 1000, 29912, 35583, 31475, 38009, 12822, 15398, 11677, 16030, 20365, 1577, 32557, 19950, 27001, 16241, 38912, 20171, 17352, 9461, 4748, 36032, 18167, 19826, 7845, 33948, 11284, 29611, 38877, 16695, 27072, 2196, 33058, 29050, 19851, 6419, 18400, 35178, 29416, 37919, 3442, 39587, 10204, 13964, 11196, 2636, 9896, 23216, 4051, 13922, 593, 14666, 1215, 38596, 17057, 14027, 399, 19838, 24521, 38945, 27537, 26960, 12158, 9379, 11056, 15940, 1038, 4783, 27817, 17475, 39190, 7632, 20945, 999, 

In [ ]:
for index, batch in enumerate(test_loader):
  r = test_indexes[index]
  print(dataframe.iloc[r]['caption'])
  break

A man in a plaid shirt uses a tool by a tree .


In [ ]:
# Testing Loop
model.eval()  # Set the model to evaluation mode
test_loss = 0
bleu_scores = []
rouge_scores = []
wer_scores = []
smoothie = SmoothingFunction().method4
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
cntr = 0

with torch.no_grad():  # Disable gradient calculation during testing
    for images, captions in test_loader:
        images, captions = images.to(device), captions.to(device)
        inputs = captions[:, :-1]
        targets = captions[:, 1:]
        outputs = model(images, inputs)

        # Calculate loss
        outputs = outputs.view(-1, outputs.size(-1))
        targets = targets.reshape(-1)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

        # Generate captions and calculate metrics
        for i in range(images.size(0)):
            oringinal_caption = dataframe.iloc[test_indexes[cntr]]['caption']#
            generated_caption = generate_caption_beam_search(images[i])
            reference_caption = tokenizer.decode(captions[i], skip_special_tokens=True)


            # Calculate BLEU-4 score




            generated_tokens = nltk.word_tokenize(generated_caption.lower())
            reference_tokens = nltk.word_tokenize(reference_caption.lower())
            bleu4 = sentence_bleu([reference_tokens], generated_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
            bleu_scores.append(bleu4)

            # Calculate ROUGE scores
            rouge = scorer.score(reference_caption, generated_caption)
            rouge_scores.append(rouge)

            # Calculate WER score
            wer = jiwer.wer(reference_caption, generated_caption)
            wer_scores.append(wer)

            if cntr < 16:
              # Plot image with captions (reduced size)
              img_np = images[i].cpu().numpy().transpose(1, 2, 0)
              img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min())
              plt.figure(figsize=(3, 3))  # Adjust size as needed
              plt.imshow(img_np)
              plt.title(f"Generated: {generated_caption}\nReference: {oringinal_caption}", fontsize=9)
              plt.axis('off')
              plt.show()
              print("-" * 50)

            cntr += 1

# Calculate average scores
avg_test_loss = test_loss / len(test_loader)
avg_bleu_score = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
avg_rouge_scores = {k: sum([s[k].fmeasure for s in rouge_scores]) / len(rouge_scores) for k in ['rouge1', 'rouge2', 'rougeL']} if rouge_scores else {k: 0 for k in ['rouge1', 'rouge2', 'rougeL']}
avg_wer_score = sum(wer_scores) / len(wer_scores) if wer_scores else 0

# Print results
print(f"Average Test Loss: {avg_test_loss:.4f}")
print(f"Average BLEU-4 Score: {avg_bleu_score:.4f}")
print(f"Average ROUGE Scores: {avg_rouge_scores}")
print(f"Average WER Score: {avg_wer_score:.4f}")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print("Metrics flickr8k CLIP LSTM No Attention")
print("="*50)
print(f"Average Training Loss: {avg_train_loss:.4f}")
print(f"Average Validation Loss: {avg_val_loss:.4f}")
print(f"Average Test Loss: {avg_test_loss:.4f}")
print(f"Average BLEU-4 Score: {avg_bleu_score:.4f}")
print(f"Average ROUGE Scores: {avg_rouge_scores}")
print(f"Average WER Score: {avg_wer_score:.4f}")

Metrics flickr8k CLIP LSTM No Attention
Average Training Loss: 0.9780
Average Validation Loss: 2.5042
Average Test Loss: 3.1001
Average BLEU-4 Score: 0.0891
Average ROUGE Scores: {'rouge1': 0.4131975146711339, 'rouge2': 0.15473539909836825, 'rougeL': 0.3817508575591527}
Average WER Score: 0.8381


In [ ]:
import gzip

In [ ]:
with open('Clip_lstm_noat_8_20_2.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# Compress the file
with open("Clip_lstm_noat_8_20_2.pkl", "rb") as f_in, gzip.open("Clip_lstm_noat_8_20_2.pkl.gz", "wb") as f_out:
    f_out.writelines(f_in)